In [39]:
import numpy as np
from scipy.linalg import expm
from tools import *

# 12 factor



Here $A, B$ are Hermitian, $a, b\in\mathbb{R}$, and
\begin{equation}
    X=[A, B], \qquad Y=i[A, [A, B]], \qquad Z=aX+bY.
\end{equation}
Then,
\begin{gather}
Z=a[A, B]+bi[A, [A, B]]=[A, aB]+i[A, [A, bB]].
\end{gather}
Here's the curve $\gamma_z (t)$ (for $t\ge 0$; and $\gamma_z (0)=I$):
\begin{equation}
    \boxed{\;
    \gamma_z (t)
    = X_t\, Y_t
    \; }
\end{equation}
with
\begin{equation}
    X_t
    = e^{\, i\sqrt{t}\, aB}\; e^{\, i \sqrt{t}\, A}\; e^{-\, i\sqrt{t}\, aB}\; e^{-\, i \sqrt{t}\, A},
\end{equation}
and
\begin{equation}
    Y_t
    = e^{\, i\, b\, t^{1/3} B}\; e^{-\, i\, t^{1/3} A}\; e^{-\, i\, b\, t^{1/3} B}\; e^{\, i\, t^{1/3} A}\;
    e^{\, i\, b\, t^{1/3} B}\; e^{\, i\, t^{1/3} A}\; e^{-\, i\, b\, t^{1/3} B}\; e^{-\, i\, t^{1/3} A}.
\end{equation}
It is straightforward to verify that $\gamma (0; x,y)=U$. The right-hand derivative of $\gamma (t; a,y)$ at $t=0$ exists and is given by
\begin{equation}
   \gamma_z^{\prime} (0)=Z.
\end{equation}





In [40]:
n = 2


A = random_hermitian(n)
B = random_hermitian(n)

X = A @ B - B @ A
Y = 1j * (A @ X - X @ A)

a = np.random.uniform(-1,1)
b = np.random.uniform(-1,1)
print(a, b)

Z = a * X + b * Y

def gamma_z(t, A, B, a, b):
    """
    Calculates the evolution operator for a quantum system using a combination of 
    Suzuki-Lie-Trotter and a retraction-like map.

    Args:
        t (float): Total evolution time.
        A (np.ndarray): First skew-Hermitian matrix operator.
        B (np.ndarray): Second skew-Hermitian matrix operator.
        a (float): First scaling coefficient.
        b (float): Second scaling coefficient.

    Returns:
        np.ndarray: The combined evolution operator.
    """
    # Calculate scaled evolution times
    assert t >= 0, "t must be non-negative"
    sqrt_t = np.sqrt(t)
    cbrt_t = np.cbrt(t)

    # Pre-calculate common exponential terms to avoid redundant computations
    exp_A_plus = expm(1j * sqrt_t * A)
    exp_A_minus = expm(-1j * sqrt_t * A)
    exp_B_plus = expm(1j * sqrt_t * a* B)
    exp_B_minus = expm(-1j * sqrt_t * a* B)

    # First component: Lie-Trotter style evolution (E1)
    Xt = exp_B_plus @ exp_A_plus @ exp_B_minus @ exp_A_minus

    # Pre-calculate common exponential terms for R
    exp_A_plus_R = expm(1j * cbrt_t * A)
    exp_A_minus_R = expm(-1j * cbrt_t * A)
    exp_B_plus_R = expm(1j * cbrt_t * b* B)
    exp_B_minus_R = expm(-1j * cbrt_t * b* B)

    # Second component: Retraction-like evolution (R)
    # The original expression for R seems to be a specific concatenation of 
    # exponential maps. We'll follow the exact sequence.
    Yt = exp_B_plus_R @ exp_A_minus_R @ exp_B_minus_R @ exp_A_plus_R @ \
        exp_B_plus_R @ exp_A_plus_R @ exp_B_minus_R @ exp_A_minus_R

    # Combine the two components
    return Xt @ Yt

t = 1e-11
gamma_prime_z = (gamma_z(t, A, B, a, b) - gamma_z(0, A, B, a, b)) / t

np.linalg.norm(gamma_prime_z - Z)



-0.24119715528573527 0.37746735465139314


8.175667227558554e-05

# 8 factor

Define the "commutator loop"

$$
\mathcal{C}(a, c ; t)=e^{i a H} e^{i c t \psi_0} e^{-i a H} e^{-i c t \psi_0} .
$$


Then, for any $x, y \in \mathbb{R}$,

$$
\gamma(t ; x, y)=\mathcal{C}\left(\frac{\pi}{2},-x ; t\right) \mathcal{C}\left(\pi, \frac{x-y}{2} ; t\right)
$$

is a product of terms of the form $e^{i \theta H}$ and $e^{i \theta \psi_0}$, satisfies $\gamma(0 ; x, y)=I$, and

$$
\gamma^{\prime}(0 ; x, y)=-x \sin \frac{\pi}{2} X_0+\left[-x\left(\cos \frac{\pi}{2}-1\right)+\frac{x-y}{2} \cdot 2(\cos \pi-1)\right] Y_0=x X_0+y Y_0 .
$$

In [41]:
# Final verification with a realistic tolerance (1e-8) reflecting central-difference + FP roundoff.
import numpy as np

rng = np.random.default_rng()

def random_unitary(d, rng):
    A = rng.normal(size=(d,d)) + 1j*rng.normal(size=(d,d))
    Q, R = np.linalg.qr(A)
    phases = np.diag(R) / np.abs(np.diag(R))
    phases[np.abs(phases) < 1e-15] = 1.0
    Q = Q @ np.diag(np.conj(phases))
    return Q

def exp_i_theta_proj(theta, P):
    d = P.shape[0]
    return np.eye(d, dtype=complex) + (np.exp(1j*theta) - 1.0)*P

def comm(A,B):
    return A@B - B@A

def make_projector_H(d, r, rng):
    U = random_unitary(d, rng)
    D = np.zeros((d,d), dtype=complex)
    D[:r, :r] = np.eye(r, dtype=complex)
    H = U @ D @ U.conj().T
    H = (H + H.conj().T)/2
    return H

def make_rank1_projector_psi0(d, rng):
    v = rng.normal(size=d) + 1j*rng.normal(size=d)
    v = v / np.linalg.norm(v)
    P = np.outer(v, np.conj(v))
    P = (P + P.conj().T)/2
    return P

def gamma_op(H, P0, x, y, t):
    a1, c1 = np.pi/2, -x
    a2, c2 = np.pi, (x - y)/2
    def C(a, c, t):
        return (
            exp_i_theta_proj(a, H) @
            exp_i_theta_proj(c*t, P0) @
            exp_i_theta_proj(-a, H) @
            exp_i_theta_proj(-c*t, P0)
        )
    return C(a1, c1, t) @ C(a2, c2, t)

def frob_norm(A):
    return np.linalg.norm(A, 'fro')

def run_trial(trial_id, d):
    r = rng.integers(1, d)
    H = make_projector_H(d, r, rng)
    P0 = make_rank1_projector_psi0(d, rng)

    X0 = comm(H, P0)
    Y0 = 1j*comm(H, X0)

    x, y = rng.normal(), rng.normal()

    eps = 1e-7
    Gm = gamma_op(H, P0, x, y, -eps)
    Gp = gamma_op(H, P0, x, y, +eps)
    G0 = gamma_op(H, P0, x, y, 0.0)

    id_err = frob_norm(G0 - np.eye(d))
    Der = (Gp - Gm)/(2*eps)
    Z = x*X0 + y*Y0
    der_err = frob_norm(Der - Z) # / max(1.0, frob_norm(Z))

    print(f"[trial {trial_id}] d={d}, rank(H)={r}, x={x:+.4f}, y={y:+.4f}")
    print(f"  ||gamma(0)-I||_F         = {id_err:.3e}")
    print(f"  rel. error in derivative = {der_err:.3e}")
    return id_err < 5e-14 and der_err < 1e-8

passes = []
trial_num = 10
for k in range(trial_num):
    d = int(rng.integers(3, 9))
    ok = run_trial(k+1, d)
    passes.append(ok)

print("\nSummary:")
print(f"  Passed {sum(passes)}/{trial_num} trials.")


[trial 1] d=3, rank(H)=2, x=+0.1953, y=-0.3765
  ||gamma(0)-I||_F         = 3.191e-15
  rel. error in derivative = 1.531e-09
[trial 2] d=4, rank(H)=1, x=-0.3325, y=-1.2092
  ||gamma(0)-I||_F         = 1.406e-15
  rel. error in derivative = 2.408e-09
[trial 3] d=5, rank(H)=3, x=+0.0088, y=+0.5633
  ||gamma(0)-I||_F         = 1.106e-15
  rel. error in derivative = 3.855e-09
[trial 4] d=3, rank(H)=1, x=+0.1392, y=+1.4549
  ||gamma(0)-I||_F         = 6.563e-16
  rel. error in derivative = 1.932e-09
[trial 5] d=8, rank(H)=1, x=-0.9071, y=-1.8629
  ||gamma(0)-I||_F         = 2.006e-15
  rel. error in derivative = 2.552e-09
[trial 6] d=4, rank(H)=3, x=+0.8985, y=-0.7373
  ||gamma(0)-I||_F         = 4.880e-15
  rel. error in derivative = 2.678e-09
[trial 7] d=8, rank(H)=4, x=-1.7047, y=+0.3772
  ||gamma(0)-I||_F         = 4.660e-15
  rel. error in derivative = 3.689e-09
[trial 8] d=5, rank(H)=2, x=-0.7149, y=+0.5393
  ||gamma(0)-I||_F         = 1.794e-15
  rel. error in derivative = 2.394e-09


# 6 factor

Let

$$
X_0=\left[H, \psi_0\right], \quad Y_0=i\left[H,\left[H, \psi_0\right]\right],
$$

and set

$$
c_1:=-\frac{x+y}{2}, \quad c_2:=\frac{x-y}{2} .
$$


Then the curve

$$
\gamma(t ; x, y)=e^{i \frac{\pi}{2} H} e^{i c_1 t \psi_0} e^{-i \pi H} e^{i c_2 t \psi_0} e^{i \frac{\pi}{2} H} e^{i y t \psi_0}
$$

uses only six occurrences of the allowed elementary exponentials and satisfies

$$
\gamma(0 ; x, y)=I, \quad \gamma^{\prime}(0 ; x, y)=x X_0+y Y_0 .
$$

In [42]:
# Correct the coefficients/signs and verify again
import numpy as np
import pandas as pd

rng = np.random.default_rng()

def random_unitary(d, rng):
    A = rng.normal(size=(d,d)) + 1j*rng.normal(size=(d,d))
    Q, R = np.linalg.qr(A)
    phases = np.diag(R) / np.abs(np.diag(R))
    phases[np.abs(phases) < 1e-15] = 1.0
    Q = Q @ np.diag(np.conj(phases))
    return Q

def projector_from_rank(d, r, rng):
    U = random_unitary(d, rng)
    D = np.zeros((d,d), dtype=complex); D[:r,:r] = np.eye(r, dtype=complex)
    H = U @ D @ U.conj().T
    return (H + H.conj().T)/2

def rank1_projector(d, rng):
    v = rng.normal(size=d) + 1j*rng.normal(size=d)
    v = v / np.linalg.norm(v)
    P = np.outer(v, np.conj(v))
    return (P + P.conj().T)/2

def exp_i_theta_proj(theta, P):
    I = np.eye(P.shape[0], dtype=complex)
    return I + (np.exp(1j*theta) - 1.0) * P

def comm(A, B): return A@B - B@A

def gamma6_correct(H, P0, x, y, t):
    # a = π/2;  c1 = -(x+y)/2,  c2 = (x - y)/2,  final factor e^{ + i y t ψ0}
    c1 = -0.5*(x + y)
    c2 =  0.5*(x - y)
    return (
        exp_i_theta_proj(np.pi/2, H) @
        exp_i_theta_proj(c1*t, P0) @
        exp_i_theta_proj(-np.pi, H) @
        exp_i_theta_proj(c2*t, P0) @
        exp_i_theta_proj(np.pi/2, H) @
        exp_i_theta_proj(y*t, P0)
    )

def frob(A): return np.linalg.norm(A, 'fro')

def verify_once(d):
    r = int(rng.integers(1, d))
    H = projector_from_rank(d, r, rng)
    P0 = rank1_projector(d, rng)
    X0 = comm(H, P0)
    Y0 = 1j*comm(H, X0)

    x, y = rng.normal(), rng.normal()
    eps = 1e-7
    Gm = gamma6_correct(H, P0, x, y, -eps)
    Gp = gamma6_correct(H, P0, x, y, +eps)
    G0 = gamma6_correct(H, P0, x, y, 0.0)

    dG = (Gp - Gm)/(2*eps)
    Z = x*X0 + y*Y0

    id_err  = frob(G0 - np.eye(d))
    der_err = frob(dG - Z) # /max(1.0, frob(Z))
    return d, r, x, y, id_err, der_err

rows = []
for _ in range(16):
    d = int(rng.integers(3, 9))
    rows.append(verify_once(d))

df = pd.DataFrame(rows, columns=["d","rank(H)","x","y","||γ(0)-I||_F","relative deriv. error"])
df


,d,rank(H),x,y,||γ(0)-I||_F,relative deriv. error
0,4,2,-0.555835,-1.697277,2.075070e-15,2.195488e-09
1,5,4,-1.255293,0.336747,2.263724e-15,2.317835e-09
2,6,1,0.891236,0.137883,8.403070e-16,3.110677e-09
3,3,1,0.783224,-0.870641,2.549838e-15,2.453107e-09
4,4,2,-1.062369,-0.008424,2.374241e-15,2.943086e-09
5,4,2,0.341966,0.570451,1.830144e-15,2.210976e-09
6,8,5,2.021272,-0.334023,2.285530e-15,4.092631e-09
7,4,2,1.611110,0.056867,6.322395e-16,2.595237e-09
8,3,2,-0.844783,0.570197,1.188740e-15,1.819373e-09
9,6,1,0.039901,-1.041450,6.519015e-16,3.770949e-09


# 5 factor

Minimal 5-factor curve
For any target $Z=x X_0+y Y_0$, set

$$
A=\operatorname{atan} 2(y, x), \quad R=\sqrt{x^2+y^2}
$$


Define

$$
a_1=A+\frac{\pi}{2}, \quad a_2=A-\frac{\pi}{2}, \quad b_1=-\frac{R}{2}, \quad b_2=+\frac{R}{2} .
$$


Then the 5-factor word

$$
\gamma_5(t ; x, y)=e^{i a_1 H} e^{i b_1 t \psi_0} e^{i\left(a_2-a_1\right) H} e^{i b_2 t \psi_0} e^{-i a_2 H}
$$

uses only $e^{i \theta H}$ and $e^{i \theta \psi_0}$, satisfies $\gamma_5(0)=I$, and

$$
\gamma_5^{\prime}(0)=x X_0+y Y_0
$$

(Here $a_2-a_1=-\pi$, so the middle $H$-factor is $e^{-i \pi H}$.)

In [43]:
# Minimal 5-factor construction and numerical verification
# ------------------------------------------------------
# For any (x,y), define A = atan2(y, x), R = sqrt(x^2 + y^2).
# Set a1 = A + π/2, a2 = A - π/2, b1 = -R/2, b2 = +R/2.
# Then:
#   γ5(t;x,y) = e^{i a1 H} e^{i b1 t ψ0} e^{i (a2 - a1) H} e^{i b2 t ψ0} e^{-i a2 H},
# and since a2 - a1 = -π, this is a 5-factor word in {e^{iθH}, e^{iθψ0}}.
# It should satisfy γ5(0)=I and γ5'(0)=x X0 + y Y0.
import numpy as np
import pandas as pd

rng = np.random.default_rng()

def random_unitary(d, rng):
    A = rng.normal(size=(d,d)) + 1j*rng.normal(size=(d,d))
    Q, R = np.linalg.qr(A)
    phases = np.diag(R) / np.abs(np.diag(R))
    phases[np.abs(phases) < 1e-15] = 1.0
    Q = Q @ np.diag(np.conj(phases))
    return Q

def projector_from_rank(d, r, rng):
    U = random_unitary(d, rng)
    D = np.zeros((d,d), dtype=complex)
    D[:r, :r] = np.eye(r, dtype=complex)
    H = U @ D @ U.conj().T
    return (H + H.conj().T)/2

def rank1_projector(d, rng):
    v = rng.normal(size=d) + 1j*rng.normal(size=d)
    v = v / np.linalg.norm(v)
    P = np.outer(v, np.conj(v))
    return (P + P.conj().T)/2

def exp_i_theta_proj(theta, P):
    I = np.eye(P.shape[0], dtype=complex)
    return I + (np.exp(1j*theta) - 1.0) * P

def comm(A, B): return A@B - B@A

def gamma5(H, P0, x, y, t):
    A = np.arctan2(y, x) if (abs(x)+abs(y))>0 else 0.0
    R = np.sqrt(x*x + y*y)
    a1, a2 = A + np.pi/2, A - np.pi/2
    b1, b2 = -0.5*R, 0.5*R
    return (
        exp_i_theta_proj(a1, H) @
        exp_i_theta_proj(b1*t, P0) @
        exp_i_theta_proj(a2 - a1, H) @  # equals exp(-iπ H)
        exp_i_theta_proj(b2*t, P0) @
        exp_i_theta_proj(-a2, H)
    )

def frob(A): return np.linalg.norm(A, 'fro')

def verify_once(d):
    r = int(rng.integers(1, d))
    H = projector_from_rank(d, r, rng)
    P0 = rank1_projector(d, rng)

    X0 = comm(H, P0)
    Y0 = 1j*comm(H, X0)

    x, y = rng.normal(), rng.normal()
    eps = 1e-7
    Gm = gamma5(H, P0, x, y, -eps)
    Gp = gamma5(H, P0, x, y, +eps)
    G0 = gamma5(H, P0, x, y, 0.0)

    dG = (Gp - Gm)/(2*eps)
    Z = x*X0 + y*Y0

    return {
        "d": d, "rank(H)": r, "x": x, "y": y,
        "||γ5(0)-I||_F": frob(G0 - np.eye(d)),
        "deriv. error": frob(dG - Z), # /max(1.0, frob(Z)),
        "||H^2-H||_F": frob(H@H - H),
        "||ψ0^2-ψ0||_F": frob(P0@P0 - P0),
    }

rows = [verify_once(int(rng.integers(3, 9))) for _ in range(20)]
df = pd.DataFrame(rows)
df


,d,rank(H),x,y,||γ5(0)-I||_F,deriv. error,||H^2-H||_F,||ψ0^2-ψ0||_F
0,4,1,-0.636375,0.465195,1.167486e-15,2.468463e-09,2.800598e-16,4.712577e-17
1,3,1,-1.625036,-0.224939,2.237090e-15,1.551443e-09,4.989253e-16,1.591228e-16
2,5,1,-1.039068,-0.270566,1.051952e-15,1.879492e-09,1.527961e-16,8.243131e-17
3,8,7,-0.844103,-1.054123,5.030990e-15,4.092304e-09,1.246276e-15,2.703561e-16
4,3,1,-1.100588,0.793853,1.226258e-15,1.090175e-09,2.322198e-16,1.834547e-16
5,6,4,0.320439,-1.415610,2.315735e-15,3.005601e-09,5.315916e-16,1.592654e-16
6,4,2,0.503751,-0.205696,2.269477e-15,2.671767e-09,5.578802e-16,1.663809e-16
7,6,2,-0.897225,-0.412961,1.583341e-15,3.675387e-09,3.410662e-16,5.060519e-17
8,8,1,-0.302135,0.029641,1.379897e-15,5.487345e-09,2.915252e-16,8.906627e-17
9,3,2,-2.312173,-0.228960,6.567435e-16,2.448476e-09,1.570092e-16,1.182932e-16


In [57]:
# Numerical verification in Python that the retraction Retr_U^{(δ)} satisfies
# γ(0) = U and (right-trivialized) γ̇(0) = η for γ(t) = Retr_U^{(δ)}(t η).
#
# Conventions:
# - We verify γ(0)=U directly.
# - For the derivative, because γ(t) ∈ SU(N), we compare the right-trivialized velocity
#   V := γ̇(0) U† with η (both lie in su(N)). We approximate γ̇(0) by a central difference.
#
# Implementation details:
# - H is an orthogonal projector: H^2 = H = H†.
# - ψ0 is a rank-1 projector: ψ0^2 = ψ0 = ψ0†, rank(ψ0)=1.
# - Define X0 = [H, ψ0], Y0 = i[H, X0]. One can check ⟨X0, Y0⟩_F = 0 and ∥X0∥_F = ∥Y0∥_F.
# - For any η in span_R{X0, Y0}, write η = α X0 + β Y0. Set
#       R = sqrt(α^2 + β^2),   A = atan2(β, α).
#   For any δ ∈ (0, 2π) \ {π}, define
#       a1 = A + δ/2,  a2 = A - δ/2,
#       b1 = -R / (2 sin(δ/2)),   b2 = +R / (2 sin(δ/2)).
#   Then
#       Retr_U^{(δ)}(η) = e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U.
# - We exploit projector exponentials: for any projector P and real θ,
#       exp(i θ P) = I + (e^{iθ} - 1) P.
#
# We run several random trials to show the identities hold numerically.
import numpy as np
import pandas as pd
from numpy.linalg import norm

rng = np.random.default_rng()

def random_unitary(n, rng):
    Z = rng.normal(size=(n,n)) + 1j*rng.normal(size=(n,n))
    Q, R = np.linalg.qr(Z)
    d = np.diag(R)
    ph = d / np.abs(d)
    U = Q * ph
    # Project to SU(n): det(U) has unit modulus; divide phase root to set det=1
    detU = np.linalg.det(U)
    U = U / detU**(1/n)
    return U

def random_projector(n, rank, rng):
    U = random_unitary(n, rng)
    D = np.zeros((n,n), dtype=complex)
    D[:rank,:rank] = np.eye(rank)
    return U @ D @ U.conj().T

def rank_one_projector(n, rng):
    v = rng.normal(size=n) + 1j*rng.normal(size=n)
    v = v / norm(v)
    return np.outer(v, np.conj(v))

def exp_i_theta_projector(P, theta):
    # exp(i θ P) = I + (e^{iθ} - 1) P, for a projector P
    return np.eye(P.shape[0], dtype=complex) + (np.exp(1j*theta) - 1.0) * P

def comm(A,B):
    return A @ B - B @ A

def retraction(U, H, psi0, A, R, delta):
    a1 = A + 0.5*delta
    a2 = A - 0.5*delta
    b1 = -R / (2.0 * np.sin(0.5*delta))
    b2 = +R / (2.0 * np.sin(0.5*delta))
    # e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U
    E1 = exp_i_theta_projector(H, a1)
    E2 = exp_i_theta_projector(psi0, b1)
    E3 = exp_i_theta_projector(H, (a2 - a1))
    E4 = exp_i_theta_projector(psi0, b2)
    E5 = exp_i_theta_projector(H, -a2)
    return (((E1 @ E2) @ (E3 @ E4)) @ E5) @ U

def innerF(A,B):
    # Frobenius inner product (real part)
    return np.trace(A.conj().T @ B).real

def to_SU_tangent(A):
    # Ensure in su(n): make skew-Hermitian and traceless (for numerical stability)
    AH = 0.5*(A - A.conj().T)  # skew-Hermitian part
    tr = np.trace(AH)/AH.shape[0]
    return AH - tr*np.eye(AH.shape[0], dtype=complex)

def run_trials(num_trials=5, n=6, eps=1e-7):
    rows = []
    for k in range(num_trials):
        # Random dimension & projector rank
        rankH = rng.integers(1, n)  # in [1, n-1]
        H = random_projector(n, rankH, rng)
        psi0 = rank_one_projector(n, rng)
        # Build X0, Y0 basis
        X0 = comm(H, psi0)                    # skew-Hermitian
        Y0 = 1j * comm(H, X0)                 # skew-Hermitian
        # Choose a random η in span{X0, Y0}
        alpha = rng.normal()
        beta  = rng.normal()
        tilde_eta = alpha * X0 + beta * Y0          # ∈ su(n)

        # Direction/magnitude parameters
        R = np.hypot(alpha, beta)             # sqrt(alpha^2 + beta^2)
        A = np.arctan2(beta, alpha)           # so that (cos A, sin A) ∥ (alpha, beta)

        # Pick δ away from 0 and π to avoid division by small sin(δ/2)
        # Exclude a band around {0, π, 2π}
        while True:
            delta = rng.uniform(0.25, 2*np.pi-0.25)  # keep away from 0 and 2π
            if np.abs(delta - np.pi) > 0.25 and np.abs(np.sin(0.5*delta)) > 1e-3:
                break

        # Random U ∈ SU(n)
        U = random_unitary(n, rng)

        # Define γ(t) = Retr_U^{(δ)}(t η). Only R scales with t.
        def gamma(t):
            return retraction(U, H, psi0, A, t*R, delta)

        # 1) Check γ(0) = U
        g0 = gamma(0.0)
        err_pos = norm(g0 - U, 'fro')

        # 2) Check right-trivialized derivative: γ̇(0) U† ≈ η
        g_pos = gamma(+eps)
        g_neg = gamma(-eps)
        dgamma = (g_pos - g_neg) / (2*eps)

        err_deriv_abs = norm(dgamma - tilde_eta @ U, 'fro')
        rel = err_deriv_abs # / max(1e-16, norm(eta_su, 'fro'))

        rows.append({
            "trial": k+1,
            "n": n,
            "rank(H)": int(rankH),
            "delta": float(delta),
            "||γ(0)-U||_F": err_pos,
            "||γ̇(0)U† - η||_F": err_deriv_abs,
            "rel_error": rel,
        })
    return pd.DataFrame(rows)

df = run_trials(num_trials=8, n=7, eps=5e-8)

df



,trial,n,rank(H),delta,||γ(0)-U||_F,||γ̇(0)U† - η||_F,rel_error
0,1,7,2,4.124957,1.763648e-15,8.074953e-09,8.074953e-09
1,2,7,2,2.652877,1.056719e-15,8.733630e-09,8.733630e-09
2,3,7,3,4.676990,1.431601e-15,9.371744e-09,9.371744e-09
3,4,7,3,1.001486,1.249115e-15,9.420614e-09,9.420614e-09
4,5,7,1,1.227184,1.126697e-15,8.453234e-09,8.453234e-09
5,6,7,2,1.597189,6.916236e-16,9.542969e-09,9.542969e-09
6,7,7,3,6.002013,9.605560e-16,7.407205e-09,7.407205e-09
7,8,7,1,3.571155,2.376476e-15,6.557854e-09,6.557854e-09


In [ ]:
# Numerical verification in Python that the retraction Retr_U^{(δ)} satisfies
# γ(0) = U and (right-trivialized) γ̇(0) = η for γ(t) = Retr_U^{(δ)}(t η).
#
# Conventions:
# - We verify γ(0)=U directly.
# - For the derivative, because γ(t) ∈ SU(N), we compare the right-trivialized velocity
#   V := γ̇(0) U† with η (both lie in su(N)). We approximate γ̇(0) by a central difference.
#
# Implementation details:
# - H is an orthogonal projector: H^2 = H = H†.
# - ψ0 is a rank-1 projector: ψ0^2 = ψ0 = ψ0†, rank(ψ0)=1.
# - Define X0 = [H, ψ0], Y0 = i[H, X0]. One can check ⟨X0, Y0⟩_F = 0 and ∥X0∥_F = ∥Y0∥_F.
# - For any η in span_R{X0, Y0}, write η = α X0 + β Y0. Set
#       R = sqrt(α^2 + β^2),   A = atan2(β, α).
#   For any δ ∈ (0, 2π) \ {π}, define
#       a1 = A + δ/2,  a2 = A - δ/2,
#       b1 = -R / (2 sin(δ/2)),   b2 = +R / (2 sin(δ/2)).
#   Then
#       Retr_U^{(δ)}(η) = e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U.
# - We exploit projector exponentials: for any projector P and real θ,
#       exp(i θ P) = I + (e^{iθ} - 1) P.
#
# We run several random trials to show the identities hold numerically.
import numpy as np
import pandas as pd
from numpy.linalg import norm

rng = np.random.default_rng()

def random_unitary(n, rng):
    Z = rng.normal(size=(n,n)) + 1j*rng.normal(size=(n,n))
    Q, R = np.linalg.qr(Z)
    d = np.diag(R)
    ph = d / np.abs(d)
    U = Q * ph
    # Project to SU(n): det(U) has unit modulus; divide phase root to set det=1
    detU = np.linalg.det(U)
    U = U / detU**(1/n)
    return U

def random_projector(n, rank, rng):
    U = random_unitary(n, rng)
    D = np.zeros((n,n), dtype=complex)
    D[:rank,:rank] = np.eye(rank)
    return U @ D @ U.conj().T

def rank_one_projector(n, rng):
    v = rng.normal(size=n) + 1j*rng.normal(size=n)
    v = v / norm(v)
    return np.outer(v, np.conj(v))

def exp_i_theta_projector(P, theta):
    # exp(i θ P) = I + (e^{iθ} - 1) P, for a projector P
    return np.eye(P.shape[0], dtype=complex) + (np.exp(1j*theta) - 1.0) * P

def comm(A,B):
    return A @ B - B @ A

def retraction(U, H, psi0, alpha, beta, delta):
    # Direction/magnitude parameters
    R = np.hypot(alpha, beta)             # sqrt(alpha^2 + beta^2)
    A = np.arctan2(beta, alpha)           # so that (cos A, sin A) ∥ (alpha, beta)
    a1 = A + 0.5*delta
    a2 = A - 0.5*delta
    b1 = -R / (2.0 * np.sin(0.5*delta))
    b2 = +R / (2.0 * np.sin(0.5*delta))
    # e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U
    E1 = exp_i_theta_projector(H, a1)
    E2 = exp_i_theta_projector(psi0, b1)
    E3 = exp_i_theta_projector(H, (a2 - a1))
    E4 = exp_i_theta_projector(psi0, b2)
    E5 = exp_i_theta_projector(H, -a2)
    return (((E1 @ E2) @ (E3 @ E4)) @ E5) @ U

def innerF(A,B):
    # Frobenius inner product (real part)
    return np.trace(A.conj().T @ B).real

def run_trials(num_trials=5, n=6, eps=1e-7):
    rows = []
    for k in range(num_trials):
        # Random dimension & projector rank
        rankH = rng.integers(1, n)  # in [1, n-1]
        H = random_projector(n, rankH, rng)
        psi0 = rank_one_projector(n, rng)
        # Build X0, Y0 basis
        X0 = comm(H, psi0)                    # skew-Hermitian
        Y0 = 1j * comm(H, X0)                 # skew-Hermitian
        # Choose a random η in span{X0, Y0}
        alpha = rng.normal()
        beta  = rng.normal()
        tilde_eta = alpha * X0 + beta * Y0          # ∈ su(n)

        # Pick δ away from 0 and π to avoid division by small sin(δ/2)
        # Exclude a band around {0, π, 2π}
        while True:
            delta = rng.uniform(0.25, 2*np.pi-0.25)  # keep away from 0 and 2π
            if np.abs(delta - np.pi) > 0.25 and np.abs(np.sin(0.5*delta)) > 1e-3:
                break

        # Random U ∈ SU(n)
        U = random_unitary(n, rng)

        # Define γ(t) = Retr_U^{(δ)}(t η). Only R scales with t.
        def gamma(t):
            return retraction(U, H, psi0, t*alpha, t*beta, delta)
        

        # 1) Check γ(0) = U
        g0 = gamma(0.0)
        err_pos = norm(g0 - U, 'fro')

        # 2) Check right-trivialized derivative: γ̇(0) U† ≈ η
        g_pos = gamma(+eps)
        g_neg = gamma(-eps)
        dgamma = (g_pos - g_neg) / (2*eps)

        err_deriv_abs = norm(dgamma - tilde_eta @ U, 'fro')
        rel = err_deriv_abs # / max(1e-16, norm(eta_su, 'fro'))

        rows.append({
            "trial": k+1,
            "n": n,
            "rank(H)": int(rankH),
            "delta": float(delta),
            "||γ(0)-U||_F": err_pos,
            "||γ̇(0)U† - η||_F": err_deriv_abs,
            "rel_error": rel,
        })
    return pd.DataFrame(rows)

df = run_trials(num_trials=8, n=7, eps=5e-8)

df

,trial,n,rank(H),delta,||γ(0)-U||_F,||γ̇(0)U† - η||_F,rel_error
0,1,7,1,1.994803,6.537550e-16,8.828487e-09,8.828487e-09
1,2,7,2,4.939021,7.813259e-16,3.052399e-08,3.052399e-08
2,3,7,6,0.678166,3.339719e-15,1.845355e-08,1.845355e-08
3,4,7,3,1.835655,3.215159e-15,2.651411e-08,2.651411e-08
4,5,7,4,2.685787,4.099643e-15,3.314823e-08,3.314823e-08
5,6,7,6,0.931384,3.158406e-15,3.815038e-08,3.815038e-08
6,7,7,1,3.867210,6.188067e-16,1.790554e-08,1.790554e-08
7,8,7,6,0.784111,4.337543e-15,6.955654e-08,6.955654e-08


In [171]:
n=7
rankH = rng.integers(1, n)  # in [1, n-1]
H = random_projector(n, rankH, rng)
psi0 = rank_one_projector(n, rng)
# Build X0, Y0 basis
X0 = comm(H, psi0)                    # skew-Hermitian
Y0 = 1j * comm(H, X0)                 # skew-Hermitian
# Choose a random η in span{X0, Y0}
U = random_unitary(n, rng)

alpha = rng.normal()
beta  = rng.normal()
tilde_eta = alpha * X0 + beta * Y0          # ∈ su(n)
eta = tilde_eta @ U

# Pick δ away from 0 and π to avoid division by small sin(δ/2)
# Exclude a band around {0, π, 2π}
while True:
    delta = rng.uniform(0.25, 2*np.pi-0.25)  # keep away from 0 and 2π
    if np.abs(delta - np.pi) > 0.25 and np.abs(np.sin(0.5*delta)) > 1e-3:
        break


# Define γ(t) = Retr_U^{(δ)}(t η). Only R scales with t.

def gamma1(t): # t eta 当做整体代入
    # Direction/magnitude parameters
    R = np.hypot(t*alpha, t*beta)             # sqrt(alpha^2 + beta^2)
    A = np.arctan2(t*beta, t*alpha)           # so that (cos A, sin A) ∥ (alpha, beta)
    a1 = A + 0.5*delta
    a2 = A - 0.5*delta
    b1 = -R / (2.0 * np.sin(0.5*delta))
    b2 = +R / (2.0 * np.sin(0.5*delta))
    # e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U
    E1 = exp_i_theta_projector(H, a1)
    E2 = exp_i_theta_projector(psi0, b1)
    E3 = exp_i_theta_projector(H, (a2 - a1))
    E4 = exp_i_theta_projector(psi0, b2)
    E5 = exp_i_theta_projector(H, -a2)
    print(R, A, a1,a2,b1,b2)
    return (((E1 @ E2) @ (E3 @ E4)) @ E5) @ U


def gamma2(t):
    # Direction/magnitude parameters
    R = np.abs(t) * np.hypot(alpha, beta)             # sqrt(alpha^2 + beta^2)
    A = np.arctan2(beta, alpha)-np.pi           # so that (cos A, sin A) ∥ (alpha, beta)
    a1 = A + 0.5*delta
    a2 = A - 0.5*delta
    b1 = -R / (2.0 * np.sin(0.5*delta))
    b2 = +R / (2.0 * np.sin(0.5*delta))
    # e^{i a1 H} e^{i b1 ψ0} e^{i (a2 - a1) H} e^{i b2 ψ0} e^{-i a2 H} U
    E1 = exp_i_theta_projector(H, a1)
    E2 = exp_i_theta_projector(psi0, b1)
    E3 = exp_i_theta_projector(H, (a2 - a1))
    E4 = exp_i_theta_projector(psi0, b2)
    E5 = exp_i_theta_projector(H, -a2)
    print(R, A, a1,a2,b1,b2)
    return (((E1 @ E2) @ (E3 @ E4)) @ E5) @ U


t = rng.normal()
print(t)
norm(gamma1(t) - gamma2(t), 'fro')

-0.8973358886040873
0.47221228857689396 -1.1544792005088504 1.8584656363688856 -4.167424037386587 -1.8403629362651102 1.8403629362651102
0.4722122885768939 -1.1544792005088502 1.8584656363688858 -4.167424037386587 -1.84036293626511 1.84036293626511


9.732093420956672e-16